So, firstly, I need to import the necesssary packages. load env variables. load openAI chat model.


Here, I will combine, Outfit attributes, skin tone, wether api data. 
and generate response based on the data using langChain-> Model, prompt, parsers, Chains and Memory

In [1]:
import os
import warnings
warnings.filterwarnings('ignore')
from dotenv import load_dotenv, find_dotenv

load_dotenv(dotenv_path=r"C:\Users\hasan\Rafi_SAA\python_practices\LLM_Langchain\llm_openai\.env")


True

In [2]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnableSequence
from langchain.chains import LLMChain
from langchain.memory import ConversationBufferMemory

In [3]:
chat = ChatOpenAI(
    model="openai/gpt-4o",
    openai_api_key=os.getenv("GPT_github_access_token"),
    openai_api_base="https://models.github.ai/inference"

)

C:\Users\hasan\AppData\Local\Temp\ipykernel_5404\722751171.py:1: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  chat = ChatOpenAI(


Taking weathear API data and filering for specific data fields.

In [6]:
import os
import requests
from dotenv import load_dotenv

# Load environment variables
load_dotenv(dotenv_path=r"C:\Users\hasan\Rafi_SAA\python_practices\LLM_Langchain\llm_openai\.env")
API_KEY = os.getenv("weather_api_key")
BASE_URL = "https://api.weatherapi.com/v1"



Current weather data for a specific city

In [7]:
def get_current_weather(city: str) -> str:
    """
    Fetch current weather for a given city.
    Returns a human-readable summary.
    """
    url = f"{BASE_URL}/current.json?key={API_KEY}&q={city}&aqi=no"
    response = requests.get(url)
    data = response.json()
    
    current = data.get("current", {})
    condition = current.get("condition", {}).get("text", "Unknown")
    temp_c = current.get("temp_c", "Unknown")
    humidity = current.get("humidity", "Unknown")
    wind_kph = current.get("wind_kph", "Unknown")
    
    return f"Current weather in {city}: {condition.lower()}, temperature around {temp_c}°C, humidity {humidity}%, wind {wind_kph} kph."


In [8]:
get_current_weather("barcelona")

'Current weather in barcelona: clear, temperature around 14.1°C, humidity 51%, wind 16.6 kph.'

Forecast data-- for a specific city.

In [ ]:
import dateparser
from datetime import datetime


def get_forecast_weather(city: str, days: str) -> str:
    """
    Fetch forecast weather for a given city.
    'days' can be 1, 3, or more depending on your plan.
    Returns a human-readable summary.
    """
    date_obj = dateparser.parse(days, settings={'PREFER_DATES_FROM': 'future'})
    # checking if user inputs wrong date
    if not date_obj:
        return "Could not determine the date. Please specify a valid date."
    
    date_str = date_obj.strftime("%Y-%m-%d")
    
    url = f"{BASE_URL}/forecast.json?key={API_KEY}&q={city}&days={date_str}&aqi=no&alerts=no"
    response = requests.get(url)
    data = response.json()
    
    forecast_day = data.get("forecast", {}).get("forecastday", [{}])[0].get("day", {})
    condition = forecast_day.get("condition", {}).get("text", "Unknown")
    avg_temp = forecast_day.get("avgtemp_c", "Unknown")
    max_temp = forecast_day.get("maxtemp_c", "Unknown")
    min_temp = forecast_day.get("mintemp_c", "Unknown")
    chance_of_rain = forecast_day.get("daily_chance_of_rain", "Unknown")
    
    return (
        f"Forecast for {city} on {date_str}: {condition.lower()}, "
        f"avg temp around {avg_temp}°C (min {min_temp}°C / max {max_temp}°C), "
        f"chance of rain {chance_of_rain}%."
    )


In [16]:
from dateparser.search import search_dates
import requests
import os
from dotenv import load_dotenv

load_dotenv(dotenv_path=r"C:\Users\hasan\Rafi_SAA\python_practices\LLM_Langchain\llm_openai\.env")
API_KEY = os.getenv("weather_api_key")
BASE_URL = "https://api.weatherapi.com/v1"

def get_forecast_weather(city: str, user_input: str) -> str:
    # Extract date from input using search_dates
    dates = search_dates(user_input, settings={'PREFER_DATES_FROM': 'future'})
    if not dates:
        return "Could not determine the date. Please specify a valid date."
    
    date_obj = dates[0][1]
    date_str = date_obj.strftime("%Y-%m-%d")
    
    # Fetch forecast
    url = f"{BASE_URL}/forecast.json?key={API_KEY}&q={city}&dt={date_str}&aqi=no&alerts=no"
    data = requests.get(url).json()
    
    forecast_day = data.get("forecast", {}).get("forecastday", [{}])[0].get("day", {})
    condition = forecast_day.get("condition", {}).get("text", "Unknown")
    avg_temp = forecast_day.get("avgtemp_c", "Unknown")
    max_temp = forecast_day.get("maxtemp_c", "Unknown")
    min_temp = forecast_day.get("mintemp_c", "Unknown")
    chance_of_rain = forecast_day.get("daily_chance_of_rain", "Unknown")
    
    return (
        f"Forecast for {city} on {date_str}: {condition.lower()}, "
        f"avg temp around {avg_temp}°C (min {min_temp}°C / max {max_temp}°C), "
        f"chance of rain {chance_of_rain}%."
    )

# Example usage
print(get_forecast_weather("Barcelona", "Barcelona on 28th October"))


Forecast for Barcelona on 2025-10-28: sunny, avg temp around 17.6°C (min 15.7°C / max 19.2°C), chance of rain 0%.


In [14]:
print(get_forecast_weather("Barcelona", "Barcelona on 28th October"))

Forecast for Barcelona tomorrow: sunny, avg temp around 19.7°C (min 17.0°C / max 21.5°C), chance of rain 0%.


In [ ]:
get_weathe